[View in Colaboratory](https://colab.research.google.com/github/tcglarry/geo/blob/master/geo_Unet_colab.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  Sample upload.txt			      Untitled1.ipynb
aoi_test	     TAHU PowerPoint Template.pdf	      Untitled2.ipynb
Capstone_PixNet.zip  TAHU PowerPoint Template - 柱狀圖 1.ods  Untitled3.ipynb
Colab Notebooks      test for excel
PiXNet		     Untitled0.ipynb


In [5]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Conv2DTranspose
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers

Using TensorFlow backend.


In [10]:
!ls drive/aoi_test/t_brain/geo

geo_Unet_colab.ipynb  train_data.npy	Untitled0.ipynb
geo_Unet.ipynb	      train_labels.npy	water_first_try.h5


In [0]:
src = 'drive/aoi_test/t_brain/geo/'

In [49]:
data= np.load(src+'train_data.npy')
label = np.load(src+'train_labels.npy')
print (np.mean(label))
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)

5.66184075511905
(4200, 21, 21, 20)
(4200, 20, 20, 1)


In [0]:
#train_data = normalize(data.reshape(4200,-1)).reshape(data.shape)



In [0]:
train_x,val_x,train_y,val_y = train_test_split(data,label,test_size=0.2, shuffle=True)

In [0]:
def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )


In [0]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=4):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((21,21,20))

    conv0 = Conv2D(32,(2,2),padding='valid')(inputs)
    print ('conv0',conv0.get_shape())
    conv1 =  conv_block(ch)(conv0)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(conv_test)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    conv7 = Conv2DTranspose(ch*4,(2,2),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
  
    model = Model(inputs= inputs, outputs=conv10)
    
    return model
    
    

In [36]:
model = build_model_1()
model.summary()

conv0 (?, 20, 20, 32)
pool1 (?, 10, 10, 4)
poo12 (?, 5, 5, 8)
conv_test (?, 4, 4, 64)
poo13 (?, 2, 2, 16)
poo14 (?, 1, 1, 32)
conv7 (?, 4, 4, 16)
conv7 (?, ?, ?, 16)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 21, 21, 20)   0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 20, 20, 32)   2592        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 20, 20, 4)    1156        conv2d_22[0][0]                  
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 20,

In [0]:



def gram_matrix(x):
  #features = K.batch_flatten(K.permute_dimensions(x[0], (2, 0, 1)))
  x = K.batch_flatten(x)
  gram = K.dot(x, K.transpose(x))
  print(gram)
  #return K.expand_dims(gram, axis=0)
  return gram
  


def my_loss(y_true,y_pred):
  diff_1  = K.mean(K.square(y_pred - y_true), axis=-1)
  diff_2 = K.mean(K.square(gram_matrix(y_pred) - gram_matrix(y_true)), axis=-1)
  
  total_loss = diff_1 * 1.0 + 0.05 * diff_2
  return total_loss
  



In [0]:
model.compile (optimizer='Adam', loss = 'mse', metrics =['mae'])

In [46]:

# train the model on the new data for a few epochs

use_model= True
if use_model:
    

    if os.path.isfile(src+'water_first_try_2.h5'):

      model = load_model(src+'water_first_try_2.h5')
      print ('model laoded')
    else:
      print ('model not exist')


model laoded


In [47]:

checkpoint = ModelCheckpoint(filepath=src+'water_first_try_2_2.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=10)
callback_list = [checkpoint, earlystop]
model.fit(x=train_x, y=train_y, batch_size=32, epochs=100, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)


Train on 2688 samples, validate on 672 samples
Epoch 1/100
2688/2688 [==============================] - 4s 1ms/step - loss: 25.5380 - val_loss: 25.8295
Epoch 2/100
2688/2688 [==============================] - 3s 1ms/step - loss: 25.9824 - val_loss: 26.7430
Epoch 3/100
2688/2688 [==============================] - 3s 985us/step - loss: 25.5748 - val_loss: 26.2475
Epoch 4/100
2688/2688 [==============================] - 3s 1ms/step - loss: 25.0840 - val_loss: 26.1634
Epoch 5/100
2688/2688 [==============================] - 3s 1ms/step - loss: 25.3493 - val_loss: 26.1062
Epoch 6/100
2688/2688 [==============================] - 3s 983us/step - loss: 25.2922 - val_loss: 26.1165
Epoch 7/100
2688/2688 [==============================] - 3s 984us/step - loss: 25.2118 - val_loss: 26.0643
Epoch 8/100
2688/2688 [==============================] - 3s 986us/step - loss: 24.8844 - val_loss: 25.5702
Epoch 9/100
2688/2688 [==============================] - 3s 1ms/step - loss: 24.9387 - val_loss: 26.0681


In [50]:
error = model.evaluate(val_x,val_y)
print (error)

840/840 [==============================] - 0s 226us/step
24.145844050816127


In [57]:

print ('label mean = ', np.mean(label))
print ('per_element_error = ', np.sqrt(error/400.))
print ('error percentage = ', str(round(np.sqrt(error/400.)/np.mean(label)*100.,2))+'%')
     

label mean =  5.66184075511905
per_element_error =  0.245692104323766
error percentage =  4.34%


In [0]:
def shuffle_list(s,l):
    final_list = []
    n_list = np.arange(l)
    np.random.shuffle(n_list)
    b_s = int(np.floor(l/s))
    #print (b_s)
    
    for i in range (b_s):
        print (i)
        final_list.append(n_list[i*s:(i+1)*s])
    return final_list

In [87]:
print (os.path.isfile('test2.npy'))
test_data = np.load('test2.npy')
test_data.shape

True


(1800, 21, 21, 20)

In [94]:
prediction = model.predict(test_data)
pred = prediction.reshape(1800,-1)
print (pred.shape)
pred_df = pd.DataFrame(pred)
pred_df.keys()


(1800, 400)


RangeIndex(start=0, stop=400, step=1)

In [0]:
test_df = pd.read_csv(src+'upload_sample.csv')


In [0]:
col_names = test_df.keys().tolist()


In [0]:
final = pd.concat([test_df['ID'],pred_df],axis=1)


In [0]:
final.columns = col_names

In [0]:
final.to_csv(src+'test_submit.csv')

In [65]:
test = os.listdir(src)
test

['train_labels.npy',
 'train_data.npy',
 'geo_Unet.ipynb',
 'water_first_try.h5',
 'geo_Unet_colab.ipynb',
 'Untitled0.ipynb',
 'water_first_try_2.h5',
 'water_first_try_2_2.h5',
 'test_data.npy']

In [0]:
l = len(train_x)
s= 32
batch_list = shuffle_list(s,l)
for batch in bacth_list:
    X = train_x[batch]
    Y = train_y[batch]
    model.train_on_batch(X,Y)
    print (np.argmax(model.predict(X),axis=-1))


NameError: name 'train_x' is not defined

In [84]:
!ls drive/aoi_test/t_brain/geo/te*


drive/aoi_test/t_brain/geo/test2.npy


In [0]:
!cp drive/aoi_test/t_brain/geo/test2.npy .

In [86]:
!ls

adc.json  drive  sample_data  test2.npy
